In [ ]:
#Connect your google drive to google colab for easier access under "Files" on the left sidebar
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Import the necessary libraries
import numpy as np
from skimage import io
import pandas as pd 
import statsmodels.api as sm

In [ ]:
#Colors Analysis part 1 (retrieving the average redness, greenness and blueness of each MRT image)

#reading the data (in excel) via pandas
df=pd.read_excel("/content/drive/MyDrive/Qualtrics_Images_url.xlsx",sheet_name="Qualtrics MRT images",header=[0])
Avg_R=[]
Avg_G=[]
Avg_B=[]

#iterating through the loaded data (from excel)
for i, j in df.iterrows():
  #retrieving each of the 3 RGB color channels in an image
  R = io.imread('{}'.format(df.iloc[i]["Image_url"]))[:, :, 0]
  G = io.imread('{}'.format(df.iloc[i]["Image_url"]))[:, :, 1]
  B = io.imread('{}'.format(df.iloc[i]["Image_url"]))[:, :, 2]
  total_R=R.sum()
  total_G=G.sum()
  total_B=B.sum()
  no_of_pixels=R.shape[0]*R.shape[1]
  avg_R=total_R/no_of_pixels
  avg_G=total_G/no_of_pixels
  avg_B=total_B/no_of_pixels
  Avg_R.append(avg_R)
  Avg_G.append(avg_G)
  Avg_B.append(avg_B)
df["Avg_R"]=Avg_R
df["Avg_G"]=Avg_G
df["Avg_B"]=Avg_B
#Export the average redness, greenness and blueness of each MRT image to another excel sheet
with pd.ExcelWriter('/content/drive/MyDrive/imgcolor.xlsx',mode='a') as writer:  
    df.to_excel(writer, sheet_name='RGB')

#References
#https://stackoverflow.com/questions/42393793/how-to-calculate-the-intensity-of-each-rgb-color-channel-of-an-image-as-a-percent
#https://stackoverflow.com/questions/51628437/compute-the-sum-of-the-red-green-and-blue-channels-using-python
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html
#https://stackoverflow.com/questions/47027254/typeerror-write-argument-must-be-str-not-bytes-python-3-vs-python-2

In [ ]:
#Colors Analysis part 1 (running linear regression model on the average redness, greenness  and blueness for all MRT images)

#reading data (from excel) using pandas 
X=pd.read_excel("/imgcolor.xlsx",sheet_name="4analysis",header=[0],index_col=[0],usecols=[0,1,2,3])
y_beautiful=pd.read_excel("/imgcolor.xlsx",sheet_name="4analysis",header=[0],index_col=[0],usecols=[0,4])
y_safe=pd.read_excel("/imgcolor.xlsx",sheet_name="4analysis",header=[0],index_col=[0],usecols=[0,5])
y_welcome=pd.read_excel("/imgcolor.xlsx",sheet_name="4analysis",header=[0],index_col=[0],usecols=[0,6])

y=[y_beautiful,y_safe,y_welcome]
R_rel_impt=[]
G_rel_impt=[]
B_rel_impt=[]
data={}
img_quality=["Beautiful","Safe","Welcome"]
#building linear regression model and printing the results 
for i in y:
    Y=i
    X = sm.add_constant(X)
    OLS_MLR = sm.OLS(Y,X)
    results = OLS_MLR.fit()
    print(results.summary())

#References
#https://datatofish.com/statsmodels-linear-regression/
#https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html#statsmodels.regression.linear_model.OLS


In [ ]:
#Colors Analysis part 2 (retrieving 64 color bins)

#creating empty 64 color bins first 

#color bin of R0G0B0 will have pixel values for each RGB channel in the range of (0-63,0-63,0-63)

#splitting the possible RGB pixel intensity value range (0 to 255) into 4 ranges
# 0-63, 64-127, 128-191, 192-255
bins=[64,128,192,256]
colors=['R','G','B']

default_value = 0
lst=[]
b0=""
b1=''
b2=""
for x in range(4):
  b0=""
  b0="R"+str(x)
  for y in range(4):
    b1=''
    b1="G"+str(y)
    for z in range(4):
      b2=""
      b2="B"+str(z)
      lst.append(b0+b1+b2)
img_bin = dict.fromkeys(lst,default_value)

pixels_bin={}
data=[]

#reading the data (from excel) using pandas
df=pd.read_excel("/content/drive/MyDrive/Qualtrics_Images_url.xlsx",sheet_name="Qualtrics MRT images",header=[0])
#iterating through the loaded data (from excel)
#populating the 64 color bins histogram for each MRT image
for i, j in df.iterrows():
  img = io.imread('{}'.format(df.iloc[i]["Image_url"]))
  for row in range(img.shape[0]):
    for col in range(img.shape[1]):
      pixel_bin_no=''
      for c,p in zip(colors,img[row][col]):
        if p<64:
          pixel_bin_no+=c+str(0)
        elif p <128:
          pixel_bin_no+=c+str(1)
        elif p <192:
          pixel_bin_no+=c+str(2)
        elif p <256:
          pixel_bin_no+=c+str(3)
      img_bin[pixel_bin_no] += 1
  data.append(img_bin)
  img_bin=img_bin.fromkeys(img_bin, 0)
df2=pd.DataFrame.from_dict(data, orient='columns')
#exporting the populated 64 color bin histogram for each image into excel sheet
with pd.ExcelWriter('/content/drive/MyDrive/imgcolor.xlsx',mode='a') as writer:  
    df2.to_excel(writer, sheet_name='64bin')

#References
#https://stackoverflow.com/questions/20079681/initializing-a-dictionary-in-python-with-a-key-value-and-no-corresponding-values
#https://codereview.stackexchange.com/questions/192739/looping-over-pixels-in-an-image
#https://www.programiz.com/python-programming/methods/built-in/zip
#https://stackoverflow.com/questions/1663807/how-to-iterate-through-two-lists-in-parallel
#https://stackoverflow.com/questions/1692388/python-list-of-dict-if-exists-increment-a-dict-value-if-not-append-a-new-dic
#https://stackoverflow.com/questions/22991888/how-to-reset-all-values-in-a-dictionary/22991916
#https://stackoverflow.com/questions/20638006/convert-list-of-dictionaries-to-a-pandas-dataframe

In [ ]:
#Colors Analysis part 2 (running correlation analysis to retrieve the most correlated color bins  (out of 64 color bins))

#read data (from excel) using pandas
df=pd.read_excel("/content/drive/MyDrive/imgcolor.xlsx",sheet_name="4analysis2",header=[0],index_col=[0])
#using pearson's correlation 
df2=df.corr(method ='pearson')
#export correlation result
with pd.ExcelWriter('/content/drive/MyDrive/imgcolor.xlsx',mode='a') as writer:  
    df2.to_excel(writer, sheet_name='64bin_corr')

In [ ]:
#Colors Analysis part 2 (running linear regression model on the most correlated color bins)

sheets=["Beautiful_bin","Safe_bin","Welcome_bin"]
for i in sheets:
  #reading data (from excel) using pandas
  X=pd.read_excel("/content/drive/MyDrive/imgcolor.xlsx",sheet_name="{}".format(i),header=[0],index_col=[0],usecols="A,C:Z")
  y=pd.read_excel("/content/drive/MyDrive/imgcolor.xlsx",sheet_name="{}".format(i),header=[0],index_col=[0],usecols=[0,1])
  Y=y
  #building linear regression model and printing the results 
  X = sm.add_constant(X)
  OLS_MLR = sm.OLS(Y,X)
  results = OLS_MLR.fit()
  print(results.summary())

#References
#https://datatofish.com/statsmodels-linear-regression/
#https://stackoverflow.com/questions/33655127/how-to-read-certain-columns-from-excel-using-pandas-python
#https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html#statsmodels.regression.linear_model.OLS